In [ ]:
# 在本机未更新至2.0.8的情况下，使用vnpy2.0.8的回测逻辑
import sys
from pathlib import Path
# new_version_path = Path(r'D:\vnpy-2.0.8')
new_version_path = Path(r'E:\vnpy\vnpy-2.0.8')
sys.path.insert(0, str(new_version_path))
# sys.path

import vnpy
print(vnpy.__version__)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('ggplot')

from datetime import datetime, timedelta
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
from double_ma_strategy import DoubleMaStrategy
from utility import vt_bar_to_df, vt_trade_to_df, load_data, strip_digt


In [ ]:
# 从数据库载入数据
price = load_data('RB888.SHFE', '1h', datetime(2019, 1, 1), datetime(2019, 12, 1))
print(f"记录数：{len(price)} 重复数：{sum(price.index.duplicated())}")
price.head()

In [ ]:
# 用matplot的交互式窗口查看数据是否正常
# 按住右键上下左右移动可以上下左右缩放，按住左键可以平移
price2 = price.reset_index()
price2.head()

%matplotlib notebook
axe = price2.close.plot(figsize=(12, 6))
axe.scatter([2, 10], [2596, 2612], color='b')
plt.show()

In [ ]:
# 设置回测参数

engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="RB888.SHFE",
    interval="1h",
    start=datetime(2019, 1, 1),
    end=datetime(2019, 12, 1),
    rate=1/10000,
    slippage=1,
    size=10,
    pricetick=1,
    capital=100000,
)
engine.add_strategy(DoubleMaStrategy, {})

In [ ]:
# 运行回测
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
df.to_csv('result.csv')
# engine.calculate_statistics()
# engine.show_chart()

In [ ]:
# 获取成交数据并在收盘价上绘制成交点
trades = engine.get_all_trades()
trade_df = vt_trade_to_df(trades)
long_df = trade_df[trade_df.direction == '多']
short_df = trade_df[trade_df.direction == '空']

# 计算成交点x,y序列
price['id'] = range(len(price))
long_x = price.loc[long_df.index].id.values
long_y = long_df.price.values
short_x = price.loc[short_df.index].id.values
short_y = short_df.price.values

# 绘制数据
%matplotlib notebook
price2 = price.reset_index()
axe = price2.close.plot(figsize=(12, 6), color='#a8a8a8', zorder=10)
axe.scatter(long_x, long_y, color='r', marker=6, zorder=20)
axe.scatter(short_x, short_y, color='g', marker=7, zorder=30)
plt.show()

In [ ]:
# 进行连续合约的回测

In [ ]:
# 进行分段合约的回测
backtest_start = datetime(2018, 3, 20)

# 载入主力合约起止日期数据
seg = pd.read_csv('dominant_data.csv', parse_dates=[1, 2])
seg['underlying'] = seg['dominant'].map(lambda x: strip_digt(x))

# 找出主力合约的起点
sel = seg[seg['underlying'] == 'RB'].copy()
sel.reset_index(inplace=True)
passed = sel[sel['start'] < backtest_start]

passed_closest_idx = passed.index.values[-1]
if passed.iloc[-1]['end'] - backtest_start < timedelta(days=7):
    passed_closest_idx += 1
sel[passed_closest_idx:]


In [ ]:
t = datetime(2019, 1, 4) - datetime(2018, 12, 4)
type(t.days)